In [1]:
labels = [
    'gal gadot', 'Jake Mcdorman', 'Keanu Reeves', 'Danielle Panabaker',
    'kiernen shipka', 'tom ellis', 'Lindsey Morgan', 'Anthony Mackie',
    'Andy Samberg', 'barack obama', 'Katherine Langford', 'Stephen Amell',
    'Brenton Thwaites', 'Sarah Wayne Callies', 'Alexandra Daddario',
    'Chris Evans', 'Sophie Turner', 'Katharine Mcphee', 'Bill Gates',
    'Mark Ruffalo', 'Gwyneth Paltrow', 'Brie Larson', 'Rami Malek',
    'Shakira Isabel Mebarak', 'Tuppence Middleton', 'Ben Affleck',
    'grant gustin', 'Taylor Swift', 'camila mendes', 'Henry Cavil',
    'Ursula Corbero', 'Richard Harmon', 'Madelaine Petsch',
    'Leonardo DiCaprio', 'Jessica Barden', 'Tom Hiddleston', 'Pedro Alonso',
    'Miley Cyrus', 'Bobby Morley', 'Anne Hathaway', 'Dominic Purcell',
    'Chris Hemsworth', 'Krysten Ritter', 'Alvaro Morte', 'Emma Stone',
    'Robert De Niro', 'Cristiano Ronaldo', 'Jeremy Renner', 'Avril Lavigne',
    'Zoe Saldana', 'Rihanna', 'Jimmy Fallon', 'Nadia Hilker', 'Tom Hardy',
    'Morgan Freeman', 'Maisie Williams', 'Dwayne Johnson', 'Irina Shayk',
    'Penn Badgley', 'Maria Pedraza', 'Robert Downey Jr', 'Natalie Portman',
    'Amanda Crew', 'Marie Avgeropoulos', 'Neil Patrick Harris',
    'elizabeth olsen', 'Elizabeth Lail', 'Wentworth Miller', 'barbara palvin',
    'Tom Cruise', 'Lionel Messi', 'Natalie Dormer', 'Hugh Jackman',
    'Chris Pratt', 'Tom Holland', 'margot robbie', 'alycia dabnem carey',
    'Emilia Clarke', 'Eliza Taylor', 'Logan Lerman', 'Jennifer Lawrence',
    'Christian Bale', 'scarlett johansson', 'amber heard', 'Zendaya',
    'Emma Watson', 'elon musk', 'Alex Lawther', 'Zac Efron', 'Selena Gomez',
    'melissa fumero', 'Millie Bobby Brown', 'Jason Momoa', 'Mark Zuckerberg',
    'Megan Fox', 'Inbar Lavi', 'Josh Radnor', 'Rebecca Ferguson',
    'Brian J. Smith', 'jeff bezos', 'ellen page', 'Adriana Lima',
    'Morena Baccarin', 'Johnny Depp', 'Lili Reinhart'
]

label_to_y = {label: i for i, label in enumerate(labels)}

In [2]:
from datasets import load_dataset
import torchvision

dataset = load_dataset('lansinuote/simple_facenet')

compose = torchvision.transforms.Compose([
    torchvision.transforms.Resize(160),
    torchvision.transforms.ToTensor(),
    lambda x: (x - 0.5) * 2,
])


def f(data):
    x = [compose(i) for i in data['image']]
    y = [label_to_y[i] for i in data['label']]
    return {'x': x, 'y': y}


dataset = dataset.with_transform(f)

dataset, dataset['train'][0]

Using custom data configuration lansinuote--simple_facenet-c1ad2d9dff6067fe
Found cached dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--simple_facenet-c1ad2d9dff6067fe/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

(DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 17334
     })
     test: Dataset({
         features: ['image', 'label'],
         num_rows: 200
     })
 }),
 {'x': tensor([[[-0.6314, -0.6314, -0.6235,  ..., -0.5686, -0.5059, -0.4431],
           [-0.6471, -0.6471, -0.6392,  ..., -0.5294, -0.4667, -0.4039],
           [-0.6863, -0.6863, -0.6784,  ..., -0.4980, -0.4353, -0.3725],
           ...,
           [-0.7725, -0.8118, -0.8431,  ...,  0.0745,  0.0745,  0.0667],
           [-0.7882, -0.8118, -0.8118,  ...,  0.0588,  0.0588,  0.0510],
           [-0.8039, -0.8039, -0.7725,  ...,  0.0510,  0.0510,  0.0510]],
  
          [[-0.4824, -0.4824, -0.4745,  ...,  0.1137,  0.1843,  0.2471],
           [-0.4902, -0.4902, -0.4824,  ...,  0.1373,  0.2078,  0.2706],
           [-0.5137, -0.5137, -0.5059,  ...,  0.1529,  0.2314,  0.2941],
           ...,
           [-0.7961, -0.8353, -0.8667,  ...,  0.0275,  0.0275,  0.0353],
           [-0.8118, -0.

In [3]:
import torch

loader_train = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=32,
    collate_fn=None,
    shuffle=True,
    drop_last=False,
)

loader_test = torch.utils.data.DataLoader(
    dataset=dataset['test'],
    batch_size=32,
    collate_fn=None,
    shuffle=True,
    drop_last=False,
)

data = next(iter(loader_train))

len(loader_train), len(loader_test), data['x'].shape, data['y']

(542,
 7,
 torch.Size([32, 3, 160, 160]),
 tensor([ 36,  57,  33,  43,  66,  79,  79,   0,  22,  52,  50,  79,   1, 101,
          10,  47,  60,  12,  74,  75,  87,  76,  32,  53,  57,  99,  78,  10,
           6,  38,   6,  98]))

In [4]:
%run 2.model.ipynb

model = Facenet.from_pretrained('lansinuote/simple_facenet')

sum(i.numel() for i in model.parameters()) / 10000

2353.6489

In [5]:
def train():
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

    model.to('cuda')
    model.train()

    correct = 0
    total = 0
    for epoch in range(20):
        for i, data in enumerate(loader_train):
            data['x'] = data['x'].to('cuda')
            data['y'] = data['y'].to('cuda')
            pred = model(data['x'])

            loss = loss_fn(pred, data['y'])
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            correct += (pred.argmax(dim=1) == data['y']).sum().item()
            total += len(data['y'])

        print(epoch, loss.item(), correct / total)
        correct = 0
        total = 0

    model.to('cpu').push_to_hub(
        repo_id='lansinuote/simple_facenet',
        token=open('/root/hub_token.txt').read().strip())


#train()

In [6]:
@torch.no_grad()
def test():
    correct = 0
    total = 0

    model.to('cpu')
    for i, data in enumerate(loader_test):
        pred = model(data['x'])

        correct += (pred.argmax(dim=1) == data['y']).sum().item()
        total += len(data['y'])

        print(i)

    print(correct / total)


test()

0
1
2
3
4
5
6
0.925
